Documentación pyedflib: https://pyedflib.readthedocs.io/en/latest/ref/edfreader.html

Documentación networkx: https://networkx.org/documentation/networkx-1.9/reference/index.html

Documentación csv: https://docs.python.org/3/library/csv.html

Información grafos: https://www.cienciadedatos.net/documentos/pygml01-introduccion-grafos-redes-python.html

In [29]:
import pandas as pd
import numpy as np
import pyedflib
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from os.path import basename
from math import sqrt
from scipy.sparse.csgraph import minimum_spanning_tree

CHANNELS: tuple[str] = ('EEG Fp1', 'EEG Fp2', 'EEG F3', 'EEG F4', 'EEG F7', 'EEG F8', 'EEG T3', 'EEG T4', 
                    'EEG C3', 'EEG C4', 'EEG T5', 'EEG T6', 'EEG P3', 'EEG P4', 'EEG O1', 'EEG O2', 
                    'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG A2-A1', 'ECG ECG')

path: str = 'Archivos\\Subject_00\\csv\\Subject00_2.csv'

# Obtiene la posicion del nombre del archivo en path y se hace un nuevo string que vaya desde 0 hasta esa posicion
path_without_filename: str = path[:path.find(basename(path))]

In [6]:
# Convertir un archivo EDF a CSV
def edf_to_csv(filename: str, output_path: str = '') -> None:
    # Se guardan los archivos en una matriz 22 x No. Señales (31000 o 91000)

    f: pyedflib.EdfReader = pyedflib.EdfReader(filename)
    eeg_channels: list[str] = f.getSignalLabels()
    n: int = len(eeg_channels)

    signals: np.zeros = np.zeros((n + 1, m := f.getNSamples()[0]))
    signals[0, :] = np.arange(1, m + 1) # Es la cabecera de las columnas que son numeros desde el 1 al 91000
    for i in np.arange(1, n + 1):
        signals[i, :]= f.readSignal(i - 1)

    np.savetxt(output_path + basename(filename.replace('.edf', '.csv')), signals, delimiter=';')

    f.close()

filename1: str = 'Archivos\\Subject_00\\edf\\Subject00_1.edf'
filename2: str = 'Archivos\\Subject_00\\edf\\Subject00_2.edf'

edf_to_csv(filename1, 'Archivos\\Subject_00\\csv\\')

In [35]:
df: pd.DataFrame = pd.read_csv(path, sep=';')

df['EEG channels'] = CHANNELS
df = df.set_index('EEG channels')

df = np.transpose(df)

# Hacer la matriz de correlación
df_correlation: pd.DataFrame = df.corr()

In [32]:
# Obtener las listas de las distancias a partir de las correlaciones
# Dij = sqrt(2 * (1 - Cij))


distances: list[list[float]] = [[sqrt(2 * (1 - df_correlation[channel][i])) for i in range(len(CHANNELS))] for channel in CHANNELS]

df_distances: pd.DataFrame = pd.DataFrame(distances, index=CHANNELS, columns=CHANNELS)


In [9]:
# Código de Andy

edges = df_distances.stack().reset_index()

edges.columns = ["asset_1", "asset_2", "distance"]


edges = edges.loc[edges["asset_1"] != edges["asset_2"]].copy()


edges

,asset_1,asset_2,distance
1,EEG Fp1,EEG Fp2,0.380983
2,EEG Fp1,EEG F3,0.384847
3,EEG Fp1,EEG F4,0.517847
4,EEG Fp1,EEG F7,0.880920
5,EEG Fp1,EEG F8,0.700337
...,...,...,...
435,ECG ECG,EEG O2,1.415922
436,ECG ECG,EEG Fz,1.389410
437,ECG ECG,EEG Cz,1.468919
438,ECG ECG,EEG Pz,1.445985


In [33]:
import csv

mst_distances = minimum_spanning_tree(df_distances)

print(mst_distances)

# Lo que se está haciendo es crear la ruta del nuevo archivo csv que guardará el minimum spanning tree, para crear la ruta se está haciendo lo siguiente:
# se utiliza la funcion basename para obtener el archivo de las señales de un sujeto, es decir, por ejemplo, 'Subject00_1.csv'
# de aqui se quita la extension del archivo con el replace ('Subject00_1') y luego se obtienen las ultimas 2 letras ('_1') con los corchetes
# finalmente se le agrega la carpeta donde se guardan los csv de cada sujeto y la extension del archivo (el .csv)

mst_path: str = path_without_filename + 'minimum_spanning_tree' + basename(path).replace('.csv', '')[-2::] + '.csv'

G_distances: nx.Graph = nx.Graph(mst_distances)
dict_distances: dict[tuple[int, int]: float] = {pair: mst_distances.data[pos] for pos, pair in enumerate(G_distances.edges())}
# Hacer un diccionario que obtenga la distancia entre cada nodo

# Se crea un archivo csv, con la informacion del mst para despues abrirlo en Gephi
with open(mst_path, 'w+', newline='') as mst_csv:
    field_names: list[str] = ['Source', 'Target', 'Weight']
    
    writer = csv.DictWriter(mst_csv, fieldnames=field_names, delimiter=';')
    
    writer.writeheader()
    for pair, distance in dict_distances.items():
        # pair es una tupla de conformada por el nodo 1 y nodo 2
        writer.writerow({'Source': pair[0], 'Target': pair[1], 'Weight': distance})

  (1, 0)	0.770004265837938
  (1, 3)	0.5323006312237416
  (2, 4)	0.7402865053906562
  (3, 2)	0.5863856132425681
  (3, 16)	0.16415034426167074
  (5, 1)	0.6693599089397927
  (6, 8)	0.781289913556856
  (7, 9)	0.7716511168959047
  (9, 3)	0.5750479331688833
  (9, 8)	0.5984842702642453
  (9, 17)	0.5949823538634543
  (11, 13)	0.6713800864020741
  (12, 10)	0.7812793468741019
  (12, 18)	0.49623267241907176
  (13, 9)	0.7647205352392905
  (13, 18)	0.5353770561333481
  (14, 12)	0.6881475452328846
  (15, 14)	0.6595006045890012
  (19, 11)	1.1821952906317017
  (19, 20)	1.241292033180945


In [34]:
# En caso de que sea necesario cambiar como se escribe el nodo al ponerse como palabra

from pprint import pprint

temp_dict: dict[tuple[int, int]: float] = {}

for pos, pair in enumerate(G_distances.edges()):
    pos1, pos2 = CHANNELS[pair[0]], CHANNELS[pair[1]]

    temp_dict[(pos1, pos2)]: float = mst_distances.data[pos]

pprint(dict_distances)
print('\n')
pprint(temp_dict)

{(0, 1): 0.770004265837938,
 (1, 3): 0.5323006312237416,
 (1, 5): 0.7402865053906562,
 (2, 3): 0.16415034426167074,
 (2, 4): 0.5863856132425681,
 (3, 9): 0.781289913556856,
 (3, 16): 0.6693599089397927,
 (6, 8): 0.7716511168959047,
 (7, 9): 0.5750479331688833,
 (8, 9): 0.5984842702642453,
 (9, 13): 0.6713800864020741,
 (9, 17): 0.5949823538634543,
 (10, 12): 0.7812793468741019,
 (11, 13): 0.49623267241907176,
 (11, 19): 0.7647205352392905,
 (12, 14): 0.6881475452328846,
 (12, 18): 0.5353770561333481,
 (13, 18): 0.6595006045890012,
 (14, 15): 1.1821952906317017,
 (19, 20): 1.241292033180945}


{('EEG A2-A1', 'ECG ECG'): 1.241292033180945,
 ('EEG C3', 'EEG C4'): 0.5984842702642453,
 ('EEG C4', 'EEG Cz'): 0.5949823538634543,
 ('EEG C4', 'EEG P4'): 0.6713800864020741,
 ('EEG F3', 'EEG F4'): 0.16415034426167074,
 ('EEG F3', 'EEG F7'): 0.5863856132425681,
 ('EEG F4', 'EEG C4'): 0.781289913556856,
 ('EEG F4', 'EEG Fz'): 0.6693599089397927,
 ('EEG Fp1', 'EEG Fp2'): 0.770004265837938,
 ('EEG Fp

In [38]:
import csv
with open(r'Archivos\Subject_00\csv\minimum_spanning_tree_2.csv', '+w', newline='') as fcsv:
    reader: csv.reader = csv.reader(fcsv, delimiter=';')

    